In [60]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/skills/skills.csv
/kaggle/input/resume-dataset/resume_data.csv


In [61]:
full_df = pd.read_csv('/kaggle/input/resume-dataset/resume_data.csv')
full_df.head()

,address,career_objective,skills,educational_institution_name,degree_names,passing_years,educational_results,result_types,major_field_of_studies,professional_company_names,...,online_links,issue_dates,expiry_dates,﻿job_position_name,educationaL_requirements,experiencere_requirement,age_requirement,responsibilities.1,skills_required,matched_score
0,NaN,Big data analytics working and database wareho...,"['Big Data', 'Hadoop', 'Hive', 'Python', 'Mapr...",['The Amity School of Engineering & Technology...,['B.Tech'],['2019'],['N/A'],[None],['Electronics'],['Coca-COla'],...,NaN,NaN,NaN,Senior Software Engineer,B.Sc in Computer Science & Engineering from a ...,At least 1 year,NaN,Technical Support\nTroubleshooting\nCollaborat...,NaN,0.850000
1,NaN,Fresher looking to join as a data analyst and ...,"['Data Analysis', 'Data Analytics', 'Business ...","['Delhi University - Hansraj College', 'Delhi ...","['B.Sc (Maths)', 'M.Sc (Science) (Statistics)']","['2015', '2018']","['N/A', 'N/A']","['N/A', 'N/A']","['Mathematics', 'Statistics']",['BIB Consultancy'],...,NaN,NaN,NaN,Machine Learning (ML) Engineer,M.Sc in Computer Science & Engineering or in a...,At least 5 year(s),NaN,Machine Learning Leadership\nCross-Functional ...,NaN,0.750000
2,NaN,NaN,"['Software Development', 'Machine Learning', '...","['Birla Institute of Technology (BIT), Ranchi']",['B.Tech'],['2018'],['N/A'],['N/A'],['Electronics/Telecommunication'],['Axis Bank Limited'],...,NaN,NaN,NaN,"Executive/ Senior Executive- Trade Marketing, ...",Master of Business Administration (MBA),At least 3 years,NaN,"Trade Marketing Executive\nBrand Visibility, S...",Brand Promotion\nCampaign Management\nField Su...,0.416667
3,NaN,To obtain a position in a fast-paced business ...,"['accounts payables', 'accounts receivables', ...","['Martinez Adult Education, Business Training ...",['Computer Applications Specialist Certificate...,['2008'],[None],[None],['Computer Applications'],"['Company Name ï¼ City , State', 'Company Name...",...,NaN,NaN,NaN,Business Development Executive,Bachelor/Honors,1 to 3 years,Age 22 to 30 years,Apparel Sourcing\nQuality Garment Sourcing\nRe...,Fast typing skill\nIELTSInternet browsing & on...,0.760000
4,NaN,Professional accountant with an outstanding wo...,"['Analytical reasoning', 'Compliance testing k...",['Kent State University'],['Bachelor of Business Administration'],[None],['3.84'],[None],['Accounting'],"['Company Name', 'Company Name', 'Company Name...",...,[None],[None],"['February 15, 2021']",Senior iOS Engineer,Bachelor of Science (BSc) in Computer Science,At least 4 years,NaN,iOS Lifecycle\nRequirement Analysis\nNative Fr...,iOS\niOS App Developer\niOS Application Develo...,0.650000


In [62]:
full_df = full_df.drop([
    'address', 'career_objective', 'professional_company_names', 'company_urls', 'start_dates',
    'end_dates', 'positions', 'locations', 'responsibilities', 'extra_curricular_activity_types',
    'extra_curricular_organization_names', 'extra_curricular_organization_links', 'role_positions', 
    'languages', 'proficiency_levels', 'certification_providers', 'online_links', 'issue_dates', 
    'expiry_dates', 'responsibilities.1'
],axis=1)

In [63]:
df

,0,1,cleaned_col1,cleaned_col2,merged_skills
0,skills,related_skils_in_job,[],[],[]
1,"['Big Data', 'Hadoop', 'Hive', 'Python', 'Mapr...",[['Big Data']],"[big data, hadoop, hive, python, mapreduce, sp...",[big data],"[big data, hadoop, hive, python, mapreduce, sp..."
2,"['Data Analysis', 'Data Analytics', 'Business ...","[['Data Analysis', 'Business Analysis', 'Machi...","[data analysis, data analytics, business analy...","[data analysis, business analysis, machine lea...","[data analysis, data analytics, business analy..."
3,"['Software Development', 'Machine Learning', '...","[['Unified Payment Interface', 'Risk Predictio...","[software development, machine learning, deep ...","[unified payment interface, risk prediction, b...","[software development, machine learning, deep ..."
4,"['accounts payables', 'accounts receivables', ...","[['accounts receivables', 'banking', 'G/L Acco...","[accounts payables, accounts receivables, acco...","[accounts receivables, banking, g/l accounts, ...","[accounts payables, accounts receivables, acco..."
...,...,...,...,...,...
9540,"['Mathematical modelling', 'Machine Learning',...","[['Data Sampling', 'Management of Agricultural...","[mathematical modelling, machine learning, pre...","[data sampling, management of agricultural pro...","[mathematical modelling, machine learning, pre..."
9541,"['Data Analysis', 'Business Analysis', 'Machin...","[['EDA', 'Data Cleaning', 'Data Modeling']]","[data analysis, business analysis, machine lea...","[eda, data cleaning, data modeling]","[data analysis, business analysis, machine lea..."
9542,"['Business Analyst', 'Data Analytics', 'Data C...",[['Machine Learning']],"[business analyst, data analytics, data cleans...",[machine learning],"[business analyst, data analytics, data cleans..."
9543,"['Machine Learning', 'Natural Language Process...","[['NLP', 'Machine Learning', 'Text Analysis']]","[machine learning, natural language processing...","[nlp, machine learning, text analysis]","[machine learning, natural language processing..."


In [64]:
full_df.columns

Index(['skills', 'educational_institution_name', 'degree_names',
       'passing_years', 'educational_results', 'result_types',
       'major_field_of_studies', 'related_skils_in_job',
       'certification_skills', '﻿job_position_name',
       'educationaL_requirements', 'experiencere_requirement',
       'age_requirement', 'skills_required', 'matched_score'],
      dtype='object')

In [65]:
full_df = full_df[['skills', 'related_skils_in_job']]


In [66]:
full_df

,skills,related_skils_in_job
0,"['Big Data', 'Hadoop', 'Hive', 'Python', 'Mapr...",[['Big Data']]
1,"['Data Analysis', 'Data Analytics', 'Business ...","[['Data Analysis', 'Business Analysis', 'Machi..."
2,"['Software Development', 'Machine Learning', '...","[['Unified Payment Interface', 'Risk Predictio..."
3,"['accounts payables', 'accounts receivables', ...","[['accounts receivables', 'banking', 'G/L Acco..."
4,"['Analytical reasoning', 'Compliance testing k...","[['collections', 'accounts receivable', 'finan..."
...,...,...
9539,"['Mathematical modelling', 'Machine Learning',...","[['Data Sampling', 'Management of Agricultural..."
9540,"['Data Analysis', 'Business Analysis', 'Machin...","[['EDA', 'Data Cleaning', 'Data Modeling']]"
9541,"['Business Analyst', 'Data Analytics', 'Data C...",[['Machine Learning']]
9542,"['Machine Learning', 'Natural Language Process...","[['NLP', 'Machine Learning', 'Text Analysis']]"


In [67]:
full_df.to_csv('skills.csv', index=False)

In [68]:
import ast

# Read the CSV file without headers
# The file contains two columns with skills data
df = pd.read_csv('/kaggle/input/skills/skills.csv', header=None, encoding='utf-8')

# Define a helper function to safely parse and clean each cell
def parse_and_clean(cell):
    try:
        # Safely convert the string representation to a Python object
        # This handles various string formats of lists
        parsed = ast.literal_eval(cell)
    except Exception as e:
        # If parsing fails, return an empty list rather than dropping the row
        return []
    
    # Define a recursive flattening function to handle nested lists
    def flatten(x):
        if isinstance(x, list):
            flat_list = []
            for item in x:
                flat_list.extend(flatten(item))
            return flat_list
        else:
            return [x]
    
    # Flatten any nested structures
    flat = flatten(parsed)
    
    # Clean each skill:
    # - Ensure it's a string
    # - Convert to lowercase for consistency
    # - Remove extra whitespace
    cleaned = [s.strip().lower() for s in flat if isinstance(s, str)]
    
    return cleaned

# Apply the cleaning function to both columns
df['cleaned_col1'] = df[0].apply(parse_and_clean)
df['cleaned_col2'] = df[1].apply(parse_and_clean)

# Merge the cleaned columns record by record
# This preserves all records, including those with empty lists
df['merged_skills'] = df.apply(lambda row: row['cleaned_col1'] + row['cleaned_col2'], axis=1)

# Remove duplicates from each merged list while preserving order
df['merged_skills'] = df['merged_skills'].apply(lambda x: list(dict.fromkeys(x)))

# Display a sample of the transformed dataframe
print(df[['cleaned_col1', 'cleaned_col2', 'merged_skills']].head())

# Save the cleaned and merged skills column for later use
# This can be added back to your original dataframe
cleaned_skills = df['merged_skills']

# Optional: Save to a file if needed
df.to_csv('cleaned_merged_skills.csv', index=False)


                                        cleaned_col1  \
0                                                 []   
1  [big data, hadoop, hive, python, mapreduce, sp...   
2  [data analysis, data analytics, business analy...   
3  [software development, machine learning, deep ...   
4  [accounts payables, accounts receivables, acco...   

                                        cleaned_col2  \
0                                                 []   
1                                         [big data]   
2  [data analysis, business analysis, machine lea...   
3  [unified payment interface, risk prediction, b...   
4  [accounts receivables, banking, g/l accounts, ...   

                                       merged_skills  
0                                                 []  
1  [big data, hadoop, hive, python, mapreduce, sp...  
2  [data analysis, data analytics, business analy...  
3  [software development, machine learning, deep ...  
4  [accounts payables, accounts receivables, acco..